In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import socket

from pyspark.sql.functions import pandas_udf
import pandas as pd
from pyspark.sql.functions import desc, col, explode, collect_list, max, when, size


import pandas as pd
import numpy as np

import re
from pyspark.sql.types import ArrayType, StringType

In [ ]:
from pyspark import SparkContext, SparkConf

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("github_csv") \
        .getOrCreate()

In [ ]:
df_new =spark.read.option("delimiter", ",").option("header", "true").option("multiline", "true").option("escape", "\\").option("escape", '"').option("quote", '"').csv("storage/GME_with_comments_groupped_text.csv")


In [ ]:
df_new.show()

+---+----------+----------+------+------------+---------+--------------------+--------------------+--------------------+----------+----------+----------+----------+----------+-----------+--------------------+
|_c0|Unnamed: 0|      date| likes|comments_num|posts_num|              titles|            comments|            selftext|      Open|      High|       Low|     Close| Adj Close|     Volume|                text|
+---+----------+----------+------+------------+---------+--------------------+--------------------+--------------------+----------+----------+----------+----------+----------+-----------+--------------------+
|  0|       435|2021-09-21|    12|        2304|       12|Update on GME AMC...| Your submission ...|      \nYour dail...|199.360001|199.360001|     186.0|189.949997|189.949997|  2633800.0|Update on GME AMC...|
|  1|       103|2020-10-06|     0|           3|        1|    PUTS ON GAMESTOP| I know most of y...|[https://www.yout...|      9.56|      9.84|       9.1|      9.13|

In [ ]:
df_emotions = spark.read.option("header", "true").csv('emojies_data/Emoji_Sentiment_Data_v1.0.csv')

In [ ]:
conds = when((col('Negative') > col('Neutral')) & (col('Negative') > col('Positive'))  , 'negative')\
            .when((col('Neutral') >= col('Negative')) & (col('Neutral') >= col('Positive'))  , 'neutral')\
            .when((col('Positive') > col('Negative')) & (col('Positive') > col('Neutral'))  , 'positive')

df_emotions = df_emotions.withColumn('Category', conds)
df_emotions.show()

+-----+-----------------+-----------+-----------+--------+-------+--------+--------------------+--------------------+--------+
|Emoji|Unicode codepoint|Occurrences|   Position|Negative|Neutral|Positive|        Unicode name|       Unicode block|Category|
+-----+-----------------+-----------+-----------+--------+-------+--------+--------------------+--------------------+--------+
|   😂|          0x1f602|      14622|0.805100583|    3614|   4163|    6845|FACE WITH TEARS O...|           Emoticons|positive|
|    ❤|           0x2764|       8050|0.746943086|     355|   1334|    6361|   HEAVY BLACK HEART|            Dingbats|positive|
|    ♥|           0x2665|       7144|0.753806008|     252|   1942|    4950|    BLACK HEART SUIT|Miscellaneous Sym...|positive|
|   😍|          0x1f60d|       6359|0.765292366|     329|   1390|    4640|SMILING FACE WITH...|           Emoticons|positive|
|   😭|          0x1f62d|       5526|0.803351976|    2412|   1218|    1896|  LOUDLY CRYING FACE|           Emotico

In [ ]:
neg_emots_list = df_emotions.where(col('Category')=='negative').select('Emoji').rdd.flatMap(lambda x: x).collect()
neu_emots_list = df_emotions.where(col('Category')=='neutral').select('Emoji').rdd.flatMap(lambda x: x).collect()
pos_emots_list = df_emotions.where(col('Category')=='positive').select('Emoji').rdd.flatMap(lambda x: x).collect()

emots_list = [*neg_emots_list, *neu_emots_list, *pos_emots_list]

In [ ]:
emots_regex = '|'.join(emots_list)

In [ ]:
emots_find = F.udf( lambda x: re.findall(emots_regex, x) if x is not None else '', returnType=ArrayType(StringType()) )

In [ ]:
emoji_dataset = df_new.select('date',
     'selftext'
    , emots_find('selftext').name('emojs')
)

emoji_dataset = emoji_dataset.withColumn('emojs_count', size(col('emojs'))).filter('emojs_count > 0')

print(emoji_dataset.printSchema())
emoji_dataset.sort(desc("emojs_count")).show()

root
 |-- date: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- emojs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- emojs_count: integer (nullable = false)

None
+----------+--------------------+--------------------+-----------+
|      date|            selftext|               emojs|emojs_count|
+----------+--------------------+--------------------+-----------+
|2021-01-28| [removed] [remov...|[🚀, 🚀, 🚀, 💎, ...|       5689|
|2021-01-27|MARKET MANIPULATI...|[💎, 💎, 🙌, 🙌, ...|       4574|
|2021-01-25|[removed]  I am a...|[🚀, 🚀, 🚀, 🚀, ...|       4231|
|2021-01-19|[removed] Listen ...|[🚀, 💎, 💎, 💎, ...|       3508|
|2021-01-17|[deleted]  [remov...|[💎, 🙌, 🚀, 🚀, ...|       3437|
|2021-01-01| &amp;#x200B;\n\n...|[🚀, 🙌, 💎, 🚀, ...|       2692|
|2021-01-22|    [removed]  [d...|[💩, 🍋, 🌚, 🚀, ...|       2256|
|2021-01-26|   [removed] [rem...|[💎, 🙌, 🚀, 🚀, ...|       1946|
|2021-01-23|[removed] [remove...|[🌈, 🐻, 🚀, 🚀, ...|       1475|
|2021-01-18| [re

In [ ]:
'''
def extract_emojs_from_column(df, column_name, emots_regex):
  emots_find =\
   F.udf( lambda x: re.findall(emots_regex, x) if x is not None else '', returnType=ArrayType(StringType()) )
  emoji_dataset = df.select(
     column_name, 
     emots_find(column_name).name('emojs'))
  emoji_dataset = emoji_dataset.withColumn('emojs_count', size(col('emojs'))).filter('emojs_count > 0')
  return emoji_dataset
'''

In [ ]:
test = extract_emojs_from_column(df_new, 'selftext', emots_regex)
test.sort(desc("emojs_count")).show()


+--------------------+--------------------+-----------+
|            selftext|               emojs|emojs_count|
+--------------------+--------------------+-----------+
| [removed] [remov...|[🚀, 🚀, 🚀, 💎, ...|       5689|
|MARKET MANIPULATI...|[💎, 💎, 🙌, 🙌, ...|       4574|
|[removed]  I am a...|[🚀, 🚀, 🚀, 🚀, ...|       4231|
|[removed] Listen ...|[🚀, 💎, 💎, 💎, ...|       3508|
|[deleted]  [remov...|[💎, 🙌, 🚀, 🚀, ...|       3437|
| &amp;#x200B;\n\n...|[🚀, 🙌, 💎, 🚀, ...|       2692|
|    [removed]  [d...|[💩, 🍋, 🌚, 🚀, ...|       2256|
|   [removed] [rem...|[💎, 🙌, 🚀, 🚀, ...|       1946|
|[removed] [remove...|[🌈, 🐻, 🚀, 🚀, ...|       1475|
| [removed] [remov...|[🚀, 🚀, 🚀, 🚀, ...|       1201|
|    Fuck you  [re...|[🚀, 🌙, 💎, 🚀, ...|        883|
| [removed]  [remo...|[🐻, 🚀, 🚀, 🚀, ...|        876|
|[removed]        ...|[💎, 🙌, 🚀, 🌙, ...|        859|
| [removed] [remov...|[🚀, 🚀, 💎, 🙌, ...|        789|
|   All you paper ...|[📄, 🙌, 🙌, 💎, ...|        784|
|[removed] [remove...|[🌈, 🐻, 🌈, 🐻, ...|        727|


In [ ]:
def count_emojs(input_df):
  emoji_dataset_emojs = input_df.select(explode('emojs').name('emojs')).groupBy('emojs').count().sort(desc('count'))
  return emoji_dataset_emojs

In [ ]:
number_emojs = count_emojs(emoji_dataset)
only_emoji= number_emojs.select("emojs")
only_emoji.show()

+-----+
|emojs|
+-----+
|   🚀|
|   💎|
|   🙌|
|   👐|
|   🐻|
|   🌈|
|    ⚫|
|   🌕|
|    ✋|
|   🍌|
|   💰|
|   🔴|
|   🌝|
|   🌙|
|   💵|
|   📈|
|   🌚|
|   🔥|
|    ⚪|
|   💥|
+-----+
only showing top 20 rows



In [ ]:
#only_emoji.toPandas().to_csv('only_emoji.csv')

In [ ]:
emoji_dataset_emojs = emoji_dataset.select(explode('emojs').name('emojs')).groupBy('emojs').count().sort(desc('count'))

In [ ]:
emoji_dataset_emojs.show()

+-----+-----+
|emojs|count|
+-----+-----+
|   🚀|38721|
|   💎| 3546|
|   🙌| 1651|
|   👐|  493|
|   🐻|  416|
|   🌈|  397|
|    ⚫|  341|
|   🌕|  313|
|    ✋|  266|
|   🍌|  234|
|   💰|  167|
|   🔴|  166|
|   🌝|  164|
|   🌙|  161|
|   💵|  154|
|   📈|  145|
|   🌚|  125|
|   🔥|  116|
|    ⚪|  112|
|   💥|  108|
+-----+-----+
only showing top 20 rows



In [ ]:
emoji_val = spark.read.option("header", "true").csv('emojies_data/emoj_val.csv')

In [ ]:
emoji_val.show()

+-----+-----+-----+
|indec|emojs|value|
+-----+-----+-----+
|    0|   🚀|    1|
|    1|   💎|    1|
|    2|   🙌|    1|
|    3|   👐|    1|
|    4|   🐻|   -1|
|    5|   🌈|    1|
|    6|    ⚫|   -1|
|    7|   🌕|    1|
|    8|    ✋|    1|
|    9|   🍌|    0|
|   10|   💰|    1|
|   11|   🔴|   -1|
|   12|   🌝|    1|
|   13|   🌙|    1|
|   14|   💵|    1|
|   15|   📈|    1|
|   16|   🌚|    1|
|   17|   🔥|    1|
|   18|    ⚪|    1|
|   19|   💥|    1|
+-----+-----+-----+
only showing top 20 rows



In [ ]:
emoji_dataset.sort(desc("emojs_count")).show()

+----------+--------------------+--------------------+-----------+
|      date|            selftext|               emojs|emojs_count|
+----------+--------------------+--------------------+-----------+
|2021-01-28| [removed] [remov...|[🚀, 🚀, 🚀, 💎, ...|       5689|
|2021-01-27|MARKET MANIPULATI...|[💎, 💎, 🙌, 🙌, ...|       4574|
|2021-01-25|[removed]  I am a...|[🚀, 🚀, 🚀, 🚀, ...|       4231|
|2021-01-19|[removed] Listen ...|[🚀, 💎, 💎, 💎, ...|       3508|
|2021-01-17|[deleted]  [remov...|[💎, 🙌, 🚀, 🚀, ...|       3437|
|2021-01-01| &amp;#x200B;\n\n...|[🚀, 🙌, 💎, 🚀, ...|       2692|
|2021-01-22|    [removed]  [d...|[💩, 🍋, 🌚, 🚀, ...|       2256|
|2021-01-26|   [removed] [rem...|[💎, 🙌, 🚀, 🚀, ...|       1946|
|2021-01-23|[removed] [remove...|[🌈, 🐻, 🚀, 🚀, ...|       1475|
|2021-01-18| [removed] [remov...|[🚀, 🚀, 🚀, 🚀, ...|       1201|
|2021-03-10|    Fuck you  [re...|[🚀, 🌙, 💎, 🚀, ...|        883|
|2021-01-24| [removed]  [remo...|[🐻, 🚀, 🚀, 🚀, ...|        876|
|2021-02-04|[removed]        ...|[💎, 🙌, 🚀, 

In [ ]:
emojs_dict = { row['emojs']: row['value'] for row in emoji_val.collect() }

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def translate(mapping):
    def translate_(col):
        return mapping(col)
    return udf(translate_, StringType())



emoji_dataset_new = emoji_dataset.withColumn("emoj_values", translate(lambda x: [emojs_dict[i] for i in x])("emojs"))

In [ ]:
emoji_dataset_new = emoji_dataset_new.withColumn("emoj_sum", translate(lambda x: sum([float(i) for i in x]))("emoj_values"))

In [ ]:
emoji_dataset_new = emoji_dataset_new.withColumn("emoj_avg", translate(lambda x: sum([float(i) for i in x])/len(x))("emoj_values"))

In [ ]:
emoji_dataset_new.show(100)

+----------+--------------------+--------------------+-----------+--------------------+--------------------+--------+
|      date|            selftext|               emojs|emojs_count|         emoj_values|            emoj_avg|emoj_sum|
+----------+--------------------+--------------------+-----------+--------------------+--------------------+--------+
|2021-01-15|      [removed] [...|[🚀, 🚀, 🚀, 🚀, ...|        702|[1, 1, 1, 1, 1, 1...|  0.9301994301994302|   653.0|
|2021-03-11|   All you paper ...|[📄, 🙌, 🙌, 💎, ...|        784|[0, 1, 1, 1, 1, 1...|  0.9107142857142857|   714.0|
|2021-01-23|[removed] [remove...|[🌈, 🐻, 🚀, 🚀, ...|       1475|[1, -1, 1, 1, 1, ...|  0.9620338983050848|  1419.0|
|2021-12-02|    I have no way...|        [💎, 💎, 💎]|          3|           [1, 1, 1]|                 1.0|     3.0|
|2021-02-25| [removed] [delet...|[🚀, 🚀, 🚀, 🚀, ...|         81|[1, 1, 1, 1, 1, 1...|  0.9876543209876543|    80.0|
|2021-02-22|   [removed] [rem...|[🚀, 🚀, 🚀, 🌈, ...|         17|[1, 1, 1, 1, 

In [ ]:
emoji_dataset_new = emoji_dataset_new.drop("selftext")

In [ ]:
emoji_dataset_new = emoji_dataset_new.drop("emoj_values")

In [ ]:
emoji_dataset_new.show()

+----------+--------------------+-----------+------------------+--------+
|      date|               emojs|emojs_count|          emoj_avg|emoj_sum|
+----------+--------------------+-----------+------------------+--------+
|2021-01-15|[🚀, 🚀, 🚀, 🚀, ...|        702|0.9301994301994302|   653.0|
|2021-03-11|[📄, 🙌, 🙌, 💎, ...|        784|0.9107142857142857|   714.0|
|2021-01-23|[🌈, 🐻, 🚀, 🚀, ...|       1475|0.9620338983050848|  1419.0|
|2021-12-02|        [💎, 💎, 💎]|          3|               1.0|     3.0|
|2021-02-25|[🚀, 🚀, 🚀, 🚀, ...|         81|0.9876543209876543|    80.0|
|2021-02-22|[🚀, 🚀, 🚀, 🌈, ...|         17|0.8823529411764706|    15.0|
|2021-02-23|[🚀, 🚀, 💎, 😄, ...|         20|              0.65|    13.0|
|2020-03-27|            [🌈, 🐻]|          2|               0.0|     0.0|
|2021-04-24|[🚀, 🚀, 🚀, 🚀, ...|         10|               1.0|    10.0|
|2021-06-21|[💎, 🙌, 💎, 🙌, ...|          9|               1.0|     9.0|
|2021-03-29|[🚀, 🚀, 🚀, 🚀, ...|         55|               1.0|    55.0|
|2021

In [ ]:
emoji_dataset_new.toPandas().to_csv('emoji_dataset_new.csv')